# Issue Hospital Credentials
## Role: NHS Trust

You should have initialised your NHS Trust agent so that it has the public keys necessary to issue Hospital credentials.

In this notebook you will connect with each of the hospital agents and issue them a Verifiable Credential attesting that they are a hospital within your Trust.

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
import nest_asyncio
from termcolor import colored, cprint

nest_asyncio.apply()
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8022
ADMIN_URL = "http://nhstrust-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


## Fetch Schema and Cred Def ID's

In [2]:
%store -r cred_def_id
%store -r schema_id
print(cred_def_id)

2rRvzQRmx8c8p4s1Demt4Z:3:CL:156569:default


## Register Listeners

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    loop = asyncio.get_event_loop()
    if STATE == 'request':
        print("ACCEPTING REQUEST")

        loop.run_until_complete(agent_controller.connections.accept_request(connection_id))
    elif STATE == 'response':
        print("SEND PING")
        time.sleep(2)
        loop.run_until_complete(agent_controller.messaging.trust_ping(connection_id, "hey"))
    elif STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))
        credential_attributes = [{"name": "name", "value": payload["their_label"]}]
        ## Offer credential
        time.sleep(1)
        loop.run_until_complete(agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False))

connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)


In [4]:
invite = await agent_controller.connections.create_invitation(multi_use="true")
print(invite['invitation'])

Connection message {'created_at': '2021-02-19 14:43:44.501234Z', 'updated_at': '2021-02-19 14:43:44.501234Z', 'connection_id': '7b54b08f-5552-44ee-9072-6dca019c75d8', 'accept': 'manual', 'state': 'invitation', 'invitation_mode': 'multi', 'routing_state': 'none', 'invitation_key': '3oTpbDd1Z3V5gdqAdKPq87k1mhi39PKE8ikhgFUPbB55', 'initiator': 'self'} 7b54b08f-5552-44ee-9072-6dca019c75d8
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'c70c44db-91b6-4b16-b100-a98e65cfae95', 'serviceEndpoint': 'http://172.17.0.1:8020', 'recipientKeys': ['3oTpbDd1Z3V5gdqAdKPq87k1mhi39PKE8ikhgFUPbB55'], 'label': 'NHS Lothian'}
Connection message {'my_did': 'C9sDcSifkWcrGWNhx4eL2i', 'created_at': '2021-02-19 14:45:27.294763Z', 'updated_at': '2021-02-19 14:45:27.294763Z', 'connection_id': '173c4264-a5b0-4ac4-ac9e-32211027fa77', 'accept': 'manual', 'state': 'invitation', 'invitation_mode': 'once', 'routing_state': 'none', 'invitation_key': '3oTpbDd1Z3V5gdqAdKPq87k1mhi39PKE8ikhg

In [ ]:
await agent_controller.terminate()